In [ ]:
import sys
from pathlib import Path
import re

import pandas as pd

# Intentar importar matplotlib / seaborn; si faltan, desactivar trazado y avisar.
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (10, 6)
    _PLOTTING_AVAILABLE = True
except Exception:
    plt = None
    sns = None
    _PLOTTING_AVAILABLE = False
    print("Aviso: matplotlib/seaborn no disponibles. Instalar con: python -m pip install matplotlib seaborn")

def _normalize_colname(col: str) -> str:
    """Normalize column name: strip, lowercase, replace spaces by underscore, remove non-word chars."""
    s = str(col).strip().lower()
    s = re.sub(r'\s+', '_', s)
    s = re.sub(r'[^\w_]', '', s)
    return s

def _normalize_columns_df(df: pd.DataFrame) -> pd.DataFrame:
    """Return DataFrame with normalized column names (non-destructive to data types)."""
    mapping = {c: _normalize_colname(c) for c in df.columns}
    return df.rename(columns=mapping)

def _find_and_rename(df: pd.DataFrame, candidates: list, target: str) -> pd.DataFrame:
    """
    If any candidate exists in df columns, rename it to target.
    Returns df (possibly modified).
    """
    for c in candidates:
        if c in df.columns:
            if c != target:
                df = df.rename(columns={c: target})
            return df
    return df

def cargar_datos(base_path: Path = Path(".")):
    """
    Carga y valida los archivos de entrada (Excel o CSV).
    Devuelve tuple: (df_clientes, df_detalle, df_productos, df_ventas) o (None, None, None, None) si falla.
    """
    files = {
        'clientes': base_path / 'clientes.xlsx',
        'detalle': base_path / 'detalle_ventas.xlsx',
        'productos': base_path / 'productos.xlsx',
        'ventas': base_path / 'ventas.xlsx',
    }

    def _read(path: Path) -> pd.DataFrame:
        if not path.exists():
            raise FileNotFoundError(f"No se encontró: {path}")
        suffix = path.suffix.lower()
        try:
            if suffix in ('.xls', '.xlsx'):
                return pd.read_excel(path, engine='openpyxl')
            else:
                try:
                    return pd.read_csv(path, encoding='utf-8')
                except UnicodeDecodeError:
                    return pd.read_csv(path, encoding='latin-1')
        except ValueError as e:
            if 'openpyxl' in str(e).lower():
                raise RuntimeError("Necesita instalar openpyxl: python -m pip install openpyxl") from e
            raise

    try:
        df_cli = _read(files['clientes'])
        df_det = _read(files['detalle'])
        df_prod = _read(files['productos'])
        df_ven = _read(files['ventas'])

        # Normalizar columnas (facilita fusiones y evita KeyError por nombres distintos)
        df_cli = _normalize_columns_df(df_cli)
        df_det = _normalize_columns_df(df_det)
        df_prod = _normalize_columns_df(df_prod)
        df_ven = _normalize_columns_df(df_ven)

        print("✅ Fase 1: Datos cargados y columnas normalizadas.\n")
        datasets = {'Clientes': df_cli, 'Detalle': df_det, 'Productos': df_prod, 'Ventas': df_ven}
        for nombre, df in datasets.items():
            nulos = int(df.isnull().sum().sum())
            estado = "Sin nulos" if nulos == 0 else f"{nulos} nulos detectados"
            print(f"{nombre}: {estado}")

        return df_cli, df_det, df_prod, df_ven

    except FileNotFoundError as e:
        print(f"❌ Error crítico: {e}")
        return None, None, None, None
    except RuntimeError as e:
        print(f"❌ Error de dependencias: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"❌ Error al leer archivos: {e}")
        return None, None, None, None

def crear_dataset_maestro(df_ven: pd.DataFrame, df_det: pd.DataFrame, df_prod: pd.DataFrame, df_cli: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza fusiones defensivas y normaliza nombres claves.
    Devuelve DataFrame maestro con columnas esperadas:
    id_venta, id_producto, id_cliente, nombre_producto, nombre_cliente, cantidad, importe, fecha, ...
    """
    if df_ven is None:
        raise ValueError("df_ven es None. Asegúrate de haber cargado los datos.")

    # Normalizar y renombrar columnas clave usando candidatos comunes
    id_venta_cands = ['id_venta','idventa','venta_id','ventaid']
    id_producto_cands = ['id_producto','idproducto','producto_id','productoid']
    id_cliente_cands = ['id_cliente','idcliente','cliente_id','clienteid']

    nombre_producto_cands = ['nombre_producto','nombre_producto','nombre','producto','name','descripcion']
    nombre_cliente_cands = ['nombre_cliente','nombre','cliente','name']

    cantidad_cands = ['cantidad','qty','unidades','cantidad_vendida','units']
    importe_cands = ['importe','monto','total','precio','valor']
    fecha_cands = ['fecha','date','fecha_venta','fecha_venta']

    # Aplicar renombrados en cada df (si se encuentra candidato)
    for df, mapping in (
        (df_det, {'id_venta': id_venta_cands, 'id_producto': id_producto_cands, 'cantidad': cantidad_cands, 'importe': importe_cands}),
        (df_ven, {'id_venta': id_venta_cands, 'id_cliente': id_cliente_cands, 'fecha': fecha_cands, 'importe': importe_cands}),
        (df_prod, {'id_producto': id_producto_cands, 'nombre_producto': nombre_producto_cands}),
        (df_cli, {'id_cliente': id_cliente_cands, 'nombre_cliente': nombre_cliente_cands}),
    ):
        for target, cands in mapping.items():
            df = _find_and_rename(df, cands, target)
        # assign back for df_det/df_ven etc (we modified local variable)
        if df is df_det:
            df_det = df
        elif df is df_ven:
            df_ven = df
        elif df is df_prod:
            df_prod = df
        elif df is df_cli:
            df_cli = df

    # Ahora verificar existencia de columnas clave y añadir fallbacks sensatos
    if 'id_venta' not in df_det.columns:
        raise KeyError("Columna 'id_venta' no encontrada en detalle de ventas.")
    if 'id_venta' not in df_ven.columns:
        raise KeyError("Columna 'id_venta' no encontrada en ventas.")

    if 'id_producto' not in df_det.columns:
        raise KeyError("Columna 'id_producto' no encontrada en detalle de ventas.")
    if 'id_producto' not in df_prod.columns:
        # si no hay id_producto en productos, intentar crear desde índice
        df_prod = df_prod.reset_index().rename(columns={'index': 'id_producto'})
        if 'id_producto' not in df_prod.columns:
            raise KeyError("Columna 'id_producto' no encontrada en productos y no se pudo inferir.")

    if 'id_cliente' not in df_ven.columns:
        raise KeyError("Columna 'id_cliente' no encontrada en ventas.")
    if 'id_cliente' not in df_cli.columns:
        # intentar crear fallback
        df_cli = df_cli.reset_index().rename(columns={'index': 'id_cliente'})
        if 'id_cliente' not in df_cli.columns:
            raise KeyError("Columna 'id_cliente' no encontrada en clientes y no se pudo inferir.")

    # Asegurar existencia de columnas cantidad/importe
    if 'cantidad' not in df_det.columns:
        if 'cantidad' in df_det.columns:
            pass
        else:
            # crear columna cantidad con 1 por defecto si no existe
            df_det['cantidad'] = 1

    if 'importe' not in df_det.columns and 'importe' not in df_ven.columns:
        raise KeyError("No se encontró columna 'importe' en detalle ni en ventas.")

    # Ejecutar merges
    master = pd.merge(df_ven, df_det, on='id_venta', how='inner', validate="1:m")
    master = pd.merge(master, df_prod, on='id_producto', how='left')
    master = pd.merge(master, df_cli, on='id_cliente', how='left', suffixes=('', '_cliente'))

    # Normalizar/asegurar tipo datetime en 'fecha'
    if 'fecha' in master.columns:
        master['fecha'] = pd.to_datetime(master['fecha'], errors='coerce')
    else:
        master['fecha'] = pd.NaT

    print(f"✅ Fase 2 completada. Dimensiones del Dataset Maestro: {master.shape}")
    return master

def analizar_productos(df: pd.DataFrame, top_n: int = 5):
    """
    Muestra los productos más vendidos por unidades e ingresos.
    Detecta automáticamente columna de nombre de producto si tiene otro nombre.
    """
    # Buscar columna de nombre producto
    name_col = next((c for c in ['nombre_producto','nombre','producto','name','descripcion'] if c in df.columns), None)
    if name_col is None:
        raise KeyError("No se encontró ninguna columna de nombre de producto en el dataset.")
    if name_col != 'nombre_producto':
        df = df.rename(columns={name_col: 'nombre_producto'})

    # Asegurar columnas numéricas
    df['cantidad'] = pd.to_numeric(df.get('cantidad', 1), errors='coerce').fillna(1)
    df['importe'] = pd.to_numeric(df.get('importe', 0), errors='coerce').fillna(0.0)

    prod_stats = df.groupby('nombre_producto').agg({'cantidad': 'sum', 'importe': 'sum'})
    top_cant = prod_stats.sort_values('cantidad', ascending=False).head(top_n)
    top_ing = prod_stats.sort_values('importe', ascending=False).head(top_n)

    print("\n📊 Top por unidades:\n", top_cant)
    print("\n📊 Top por ingresos:\n", top_ing)

    if _PLOTTING_AVAILABLE:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        sns.barplot(x=top_cant['cantidad'], y=top_cant.index, ax=axes[0], palette='viridis')
        axes[0].set_title(f'Top {top_n} Productos más Vendidos (Unidades)')
        axes[0].set_xlabel('Unidades Vendidas')
        axes[0].set_ylabel('')

        sns.barplot(x=top_ing['importe'], y=top_ing.index, ax=axes[1], palette='magma')
        axes[1].set_title(f'Top {top_n} Productos por Ingresos ($)')
        axes[1].set_xlabel('Total Generado ($)')
        axes[1].set_ylabel('')

        plt.tight_layout()
        plt.show()
    else:
        print("Gráficos omitidos: matplotlib/seaborn no disponibles.")

def analizar_tendencia_temporal(df: pd.DataFrame):
    """
    Agrupa por mes-año y muestra la tendencia de ingresos mensuales.
    """
    if 'fecha' not in df.columns:
        print("No hay columna 'fecha' para análisis temporal.")
        return
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
    df = df.dropna(subset=['fecha'])
    if df.empty:
        print("No hay fechas válidas para análisis temporal.")
        return

    df['mes_año'] = df['fecha'].dt.to_period('M')
    df['importe'] = pd.to_numeric(df.get('importe', 0), errors='coerce').fillna(0.0)
    ventas_mes = df.groupby('mes_año')['importe'].sum()
    print("\n📈 Ventas por mes:\n", ventas_mes)

    if _PLOTTING_AVAILABLE:
        x_labels = ventas_mes.index.astype(str)
        plt.figure(figsize=(12, 6))
        sns.lineplot(x=x_labels, y=ventas_mes.values, marker='o', linewidth=3, color='#2ecc71')
        plt.title('Tendencia de Ingresos Mensuales')
        plt.ylabel('Ventas Totales ($)')
        plt.xlabel('Mes')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        for x, y in zip(x_labels, ventas_mes.values):
            plt.text(x, y, f"${y:,.0f}", ha='center', va='bottom', fontsize=9)
        plt.show()
    else:
        print("Gráficos omitidos: matplotlib/seaborn no disponibles.")

def analizar_mejores_clientes(df: pd.DataFrame, top_n: int = 5):
    """
    Identifica los clientes con mayor gasto acumulado.
    """
    name_col = next((c for c in ['nombre_cliente','cliente','nombre','name'] if c in df.columns), None)
    if name_col is None:
        print("No se encontró columna de nombre de cliente. Se usará 'id_cliente' como etiqueta.")
        if 'id_cliente' not in df.columns:
            raise KeyError("No hay columna para identificar clientes.")
        df['nombre_cliente'] = df['id_cliente'].astype(str)
    else:
        if name_col != 'nombre_cliente':
            df = df.rename(columns={name_col: 'nombre_cliente'})

    df['importe'] = pd.to_numeric(df.get('importe', 0), errors='coerce').fillna(0.0)
    top_clientes = df.groupby('nombre_cliente')['importe'].sum().sort_values(ascending=False).head(top_n)
    print("\n👥 Top clientes por gasto:\n", top_clientes)

    if _PLOTTING_AVAILABLE:
        plt.figure(figsize=(10, 5))
        sns.barplot(x=top_clientes.values, y=top_clientes.index, palette='coolwarm')
        plt.title(f'Top {top_n} Clientes con Mayor Volumen de Compra')
        plt.xlabel('Gasto Total Acumulado ($)')
        plt.ylabel('Cliente')
        plt.show()
    else:
        print("Gráficos omitidos: matplotlib/seaborn no disponibles.")

# --- EJECUCIÓN PRINCIPAL ---
def main():
    base_path = Path(".")
    df_clientes, df_detalle, df_productos, df_ventas = cargar_datos(base_path)

    if df_clientes is None:
        print("Finalizando: no se cargaron datos.")
        return

    df_maestro = crear_dataset_maestro(df_ventas, df_detalle, df_productos, df_clientes)
    analizar_productos(df_maestro)
    analizar_tendencia_temporal(df_maestro)
    analizar_mejores_clientes(df_maestro)

if __name__ == "__main__":
    main()
```# filepath: c:\Users\Miguel\Desktop\Miguelon\GH\Formacion_IA_DataScience_ML\Sprint-2\Data\Tienda_Aurelion.ipynb
import sys
from pathlib

✅ Fase 1: Datos cargados exitosamente.

--- Reporte rápido de valores nulos ---
Clientes: Sin nulos
Detalle: Sin nulos
Productos: Sin nulos
Ventas: Sin nulos
✅ Fase 2 completada. Dimensiones del Dataset Maestro: (343, 18)

📊 Iniciando análisis de productos...


KeyError: 'nombre_producto'